## Topological Sorting

To demonstrate that computer scientists can turn just about anything into a graph problem, let’s consider the difficult problem of stirring up a batch of pancakes. The recipe is really quite simple: 1 egg, 1 cup of pancake mix, 1 tablespoon oil, and $\frac{3}{4}$ cup of milk. To make pancakes you must heat the griddle, mix all the ingredients together and spoon the mix onto a hot griddle. When the pancakes start to bubble you turn them over and let them cook until they are golden brown on the bottom. Before you eat your pancakes you are going to want to heat up some syrup. The next Figure illustrates this process as a graph.

![](./images/ts1.png)

The difficult thing about making pancakes is knowing what to do first. As you can see from the previews Figure, you might start by heating the griddle or by adding any of the ingredients to the pancake mix. To help us decide the precise order in which we should do each of the steps required to make our pancakes we turn to a graph algorithm called the **topological sort**.

A topological sort takes a **directed acyclic graph** and produces a linear ordering of all its vertices such that if the graph $G$ contains an edge $(v,w)$ then the vertex $v$ comes before the vertex $w$ in the ordering. Directed acyclic graphs are used in many applications to indicate the precedence of events. Making pancakes is just one example; other examples include software project schedules, precedence charts for optimizing database queries, linear algebra operations in artificial intelligence pipelines and sequences of matrix multiplications.

The topological sort is a simple but useful adaptation of a depth first search. The algorithm for the topological sort is as follows:

1. Call `dfs(g)` for some graph `g`. The main reason we want to call depth first search is to compute the finish times for each of the vertices.
2. Store the vertices in a list in decreasing order of finish time.
3. Return the ordered list as the result of the topological sort.

The next Figure shows the depth first forest constructed by `dfs` on the pancake-making graph shown in the previous Figure.

![](./images/ts2.png)

Finally, the next Figure shows the results of applying the topological sort algorithm to our graph. Now all the ambiguity has been removed and we know exactly the order in which to perform the pancake making steps.

![](./images/ts3.png)

In code:

In [2]:
from utils import Graph, Vertex
import operator              
    
class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0

    def dfs(self):
        for aVertex in self:
            aVertex.setColor('white')
            aVertex.setPred(-1)
        for aVertex in self:
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex)

    def dfsvisit(self,startVertex):
        startVertex.setColor('gray')
        self.time += 1
        startVertex.setDiscovery(self.time)
        for nextVertex in startVertex.getConnections():
            if nextVertex.getColor() == 'white':
                nextVertex.setPred(startVertex)
                self.dfsvisit(nextVertex)
        startVertex.setColor('black')
        self.time += 1
        startVertex.setFinish(self.time)

Next, let's formalize the graph with the recipe for making pancakes:

In [4]:
g=DFSGraph()

g.addEdge('cupMilk','cupMix')
g.addEdge('egg','cupMix')
g.addEdge('Tbl','cupMix')
g.addEdge('cupMix','heatSyrup')
g.addEdge('cupMix','pour')
g.addEdge('heatGriddle','pour')
g.addEdge('pour','turn')
g.addEdge('turn','eat')
g.addEdge('heatSyrup','eat')

now we apply the algorithm for topological sorting

In [6]:
#1. Call depth first search dfs(g) for some graph g to compute the finish times for each of the vertyices.

g.dfs()

#2 Store the vertices in a list in decreasing order of finish time.
triplets = []
for v in g:
    triplets.append((v.id, v.disc, v.fin))
triplets    

sortedTriplets = sorted(triplets, key=lambda tup: tup[2], reverse=True)


#Return the ordered list as the result of the topological sort.

for i, e in enumerate(sortedTriplets):
    #print(i, e, end=",")
    print(i, e)

0 ('heatGriddle', 35, 36)
1 ('Tbl', 33, 34)
2 ('egg', 31, 32)
3 ('cupMilk', 19, 30)
4 ('cupMix', 20, 29)
5 ('pour', 25, 28)
6 ('turn', 26, 27)
7 ('heatSyrup', 21, 24)
8 ('eat', 22, 23)


## Strongly Connected Components

Many problems in real life deal with extremely large graphs. The graphs we will use to study some additional algorithms are the graphs produced by the connections between hosts on the Internet and the links between web pages. We will begin with web pages.

Search engines like Google and Bing exploit the fact that the pages on the web form a very large directed graph. To transform the World Wide Web into a graph, we will treat a page as a vertex, and the hyperlinks on the page as edges connecting one vertex to another. The next Figure shows a very small part of the graph produced by following the links from one page to the next, beginning at a university’s Computer Science home page. Of course, this graph could be huge, so for visualization it has been limited to web sites that are no more than 10 links away from the original computer science home page.

![](./images/scc1.png)

If you study the previous graph you might make some interesting observations. First you might notice that many of the other web sites on the graph are other Luther College web sites. Second, you might notice that there are several links to other colleges in Iowa. Third, you might notice that there are several links to other liberal arts colleges. You might conclude from this that there is some underlying structure to the web that clusters together web sites that are similar on some level.

One graph algorithm that can help find clusters of highly interconnected vertices in a graph is called the **strongly connected components algorithm (SCC)**. We formally define a strongly connected component, $C$, of a graph $G$, as the largest subset of vertices $C\subset V$ such that for every pair of vertices $v,w \in C$ we have a path from $v$ to $w$ and a path from $w$ to $v$. The next Figure shows a simple graph with three strongly connected components. The strongly connected components are identified by the different shaded areas.

![](./images/scc2.png)

Once the strongly connected components have been identified we can show a simplified view of the graph by combining all the vertices in one strongly connected component into a single larger vertex. The simplified version of the graph in the previous Figure is shown below.

![](./images/scc3.png)

Once again we will see that we can create a very powerful and efficient algorithm by making use of a depth first search. Before we tackle the main **SCC** algorithm we must look at one other definition. The transposition of a graph $G$ is defined as the graph $G^T$ where all the edges in the graph have been reversed. That is, if there is a directed edge from node A to node B in the original graph then $G^T$ will contain and edge from node B to node A. The next Figure shows a simple graph and its transposition.

![](./images/scc4.png)

Notice that both $G$ and $G^T$ have the same two strongly connected components.

We can now describe the algorithm to compute the strongly connected components for a graph.

1. Call `dfs` for the graph $G$ to compute the finish times for each vertex.
2. Compute $G^T$.
3. Call `dfs` for the graph $G^T$ but in the main loop of DFS explore each vertex in decreasing order of finish time.
4. Each tree in the forest computed in step 3 is a strongly connected component. Output the vertex ids for each vertex in each tree in the forest to identify the component.

Let’s trace the operation of the steps described above on the example graph on the left of the next Figure. The Figure in the middle shows the starting and finishing times computed for the original graph $G$ by the DFS algorithm. The right Figure shows the starting and finishing times computed by running DFS on the transposed graph $G^T$.

![](./images/scc5.png)

Finally, the next Figure shows the forest of three trees produced in step 3 of the strongly connected component algorithm. 

![](./images/scc6.png)

In code:

In [17]:
from utils import Graph, Vertex
import operator              
    
class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0

    def dfs(self):
        for aVertex in self:
            aVertex.setColor('white')
            aVertex.setPred(-1)
        for aVertex in self:
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex)

    def dfsvisit(self,startVertex):
        startVertex.setColor('gray')
        self.time += 1
        startVertex.setDiscovery(self.time)
        for nextVertex in startVertex.getConnections():
            if nextVertex.getColor() == 'white':
                nextVertex.setPred(startVertex)
                self.dfsvisit(nextVertex)
        startVertex.setColor('black')
        self.time += 1
        startVertex.setFinish(self.time)
        
    def dfsDecreasingOrderOfFinishTime(self):
        for aVertex in self:
            aVertex.setColor('white')
            aVertex.setPred(-1)
        for aVertex in (sorted(self.vertices.values(), key=operator.attrgetter('fin'),reverse=True)):
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex)  

We define the original graph $G$:

In [18]:
g = DFSGraph()

g.addEdge('a','b')
g.addEdge('b','c')
g.addEdge('b','e')
g.addEdge('c','c')
g.addEdge('c','f')
g.addEdge('d','b')
g.addEdge('d','g')
g.addEdge('e','a')
g.addEdge('e','d')
g.addEdge('f','h')
g.addEdge('g','e')
g.addEdge('h','i')
g.addEdge('i','f')

Next, we apply the algorithm for finding the strongly connected components in graph $G$:

In [19]:
#1. Call dfs for the graph  GG  to compute the finish times for each vertex.

g.dfs()

for v in g:
    print("Node: " + v.id + ". Color: " + v.color +  ". Discovery time: " + str(v.disc) + ". Finished time: " + str(v.fin))

Node: a. Color: black. Discovery time: 1. Finished time: 18
Node: b. Color: black. Discovery time: 2. Finished time: 17
Node: c. Color: black. Discovery time: 3. Finished time: 10
Node: e. Color: black. Discovery time: 11. Finished time: 16
Node: f. Color: black. Discovery time: 4. Finished time: 9
Node: d. Color: black. Discovery time: 12. Finished time: 15
Node: g. Color: black. Discovery time: 13. Finished time: 14
Node: h. Color: black. Discovery time: 5. Finished time: 8
Node: i. Color: black. Discovery time: 6. Finished time: 7


![](./images/scc5.png)

In [21]:
#2. Compute  gT

gT = DFSGraph()

gT.addEdge('b','a')
gT.addEdge('c','b')
gT.addEdge('e','b')
gT.addEdge('c','c')
gT.addEdge('f','c')
gT.addEdge('b','d')
gT.addEdge('g','d')
gT.addEdge('a','e')
gT.addEdge('d','e')
gT.addEdge('h','f')
gT.addEdge('e','g')
gT.addEdge('i','h')
gT.addEdge('f','i')     

#Copy the finish times for each vertex learned from dfs() to gT
for v in g:
    gT.getVertex(v.id).fin = v.fin

In [22]:
#3. Call dfs for the graph  GTGT  but in the main loop of DFS explore each vertex in decreasing order of finish time.

gT.dfsDecreasingOrderOfFinishTime()

In [23]:
#4. Each tree in the forest computed in step 3 is a strongly connected component. Output the vertex ids for each vertex in each tree in the forest to identify the component.
for v in gT:
    print("Node: " + v.id + ". Color: " + v.color +  ". Discovery time: " + str(v.disc) + ". Finished time: " + str(v.fin))

Node: b. Color: black. Discovery time: 3. Finished time: 6
Node: a. Color: black. Discovery time: 1. Finished time: 10
Node: c. Color: black. Discovery time: 11. Finished time: 12
Node: e. Color: black. Discovery time: 2. Finished time: 9
Node: f. Color: black. Discovery time: 13. Finished time: 18
Node: d. Color: black. Discovery time: 4. Finished time: 5
Node: g. Color: black. Discovery time: 7. Finished time: 8
Node: h. Color: black. Discovery time: 15. Finished time: 16
Node: i. Color: black. Discovery time: 14. Finished time: 17


![](./images/scc6.png)

## Shortest Path Problems

When you surf the web, send an email, or log in to a laboratory computer from another location on campus a lot of work is going on behind the scenes to get the information on your computer transferred to another computer. The in-depth study of how information flows from one computer to another over the Internet is the primary topic for a class in computer networking. However, we will talk about how the Internet works just enough to understand another very important graph algorithm.

![](./images/sp1.png)

The previous Figure shows you a high-level overview of how communication on the Internet works. When you use your browser to request a web page from a server, the request must travel over your local area network and out onto the Internet through a router. The request travels over the Internet and eventually arrives at a router for the local area network where the server is located. The web page you requested then travels back through the same routers to get to your browser. Inside the cloud labelled “Internet” in the Figure above are additional routers. The job of all of these routers is to work together to get your information from place to place. You can see there are many routers for yourself if your computer supports the `traceroute` (UNIX) or `tracert` (Windows) command. The text below shows the output of the `tracert` command between a user computer and the web domain [http://howtogeek.com](http://howtogeek.com) which illustrates that there are 14 routers between Chris' computer and the Web server at [http://howtogeek.com](http://howtogeek.com).

![](./images/sp2.png)

Each router on the Internet is connected to one or more other routers. So if you run the `traceroute/tracert` command at different times of the day, you are likely to see that your information flows through different routers at different times. This is because there is a cost associated with each connection between a pair of routers that depends on the volume of traffic, the time of day, and many other factors. By this time it will not surprise you to learn that we can represent the network of routers as a graph with weighted edges.

![](./images/sp3.png)

The previous Figure shows a small example of a weighted graph that represents the interconnection of routers in the Internet. The problem that we want to solve is to find the path with the smallest total weight along which to route any given message. This problem should sound familiar because it is similar to the problem we solved using a breadth first search, except that here we are concerned with the total weight of the path rather than the number of hops in the path. It should be noted that if all the weights are equal, the problem is the same.

## Dijkstra’s Algorithm

The algorithm we are going to use to determine the shortest path is called **Dijkstra’s algorithm**. Dijkstra’s algorithm is an iterative algorithm that provides us with the shortest path from one particular starting node to all other nodes in the graph. Again this is similar to the results of a breadth first search.

To keep track of the total cost from the start node to each destination we will make use of the `dist` instance variable in the Vertex class. The `dist` instance variable will contain the current total weight of the smallest weight path from the start vertex to the current vertex. The algorithm iterates once for every vertex in the graph; however, the order that we iterate over the vertices is controlled by a priority queue. The value that is used to determine the order of the objects in the priority queue is `dist`. When a vertex is first created `dist` is set to a very large number. Theoretically you would set `dist` to infinity, but in practice we just set it to a number that is larger than any real distance we would have in the problem we are trying to solve.

The code for Dijkstra’s algorithm is shown in the code snippet below. When the algorithm finishes the distances are set correctly as are the predecessor links for each vertex in the graph.

In [1]:
from utils import PriorityQueue, Graph, Vertex

def dijkstra(aGraph,start):
    pq = PriorityQueue()
    start.setDistance(0)
    pq.buildHeap([(v.getDistance(),v) for v in aGraph])
    while not pq.isEmpty():
        currentVert = pq.delMin()
        for nextVert in currentVert.getConnections():
            newDist = currentVert.getDistance() + currentVert.getWeight(nextVert)
            if newDist < nextVert.getDistance():
                nextVert.setDistance( newDist )
                nextVert.setPred(currentVert)
                pq.decreaseKey(nextVert,newDist)

Dijkstra’s algorithm uses a priority queue. You may recall that a priority queue is based on the heap that we implemented in the Tree Chapter. There are a couple of differences between that simple implementation and the implementation we use for Dijkstra’s algorithm. First, the `PriorityQueue` class stores tuples of key, value pairs. This is important for Dijkstra’s algorithm as the key in the priority queue must match the key of the vertex in the graph. Secondly the value is used for deciding the priority, and thus the position of the key in the priority queue. In this implementation we use the distance to the start vertex as the priority because as we will see when we are exploring the next vertex, we always want to explore the vertex that has the smallest distance. The second difference is the addition of the `decreaseKey` method. As you can see, this method is used when the distance to a vertex that is already in the queue is reduced, and thus moves that vertex toward the front of the queue.

Let’s walk through an application of Dijkstra’s algorithm one vertex at a time using the following sequence of figures as our guide. We begin with the vertex $u$ as start vertex. The three vertices adjacent to $u$ are $v$,$w$, and $x$. Since the initial distances to $v$,$w$, and $x$ are all initialized to `sys.maxint`, the new costs to get to them through the start node are all their direct costs. So we update the costs to each of these three nodes. We also set the predecessor for each node to $u$ (indicated in the figure with dashed curves) and we add each node to the priority queue. We use the distance as the key for the priority queue. The state of the algorithm at this stage is shown in Figure a) below.

In the next iteration of the while loop b) the vertex $x$ is next because it has the lowest overall cost and therefore bubbled its way to the beginning of the priority queue. We examine the vertices that are adjacent (neighbors) to $x$:  $u$,$v$,$w$ and $y$. For each neighboring vertex we check to see if the distance to that vertex through $x$ is smaller than the previously known distance. Obviously this is the case for $y$ since its distance was `sys.maxint`. It is not the case for $u$ or $v$ since their distances are 0 and 2 respectively. However, we now learn that the distance from the start vertex $u$ to $w$ is smaller if we go through $x$ than from $u$ directly to $w$. Since that is the case we update $w$ with a new distance and change the predecessor for $w$ from $u$ to $x$. See Figure b) for the state of all the vertices at this stage of the algorithm.

The next step is to look at the vertices neighboring $v$ (the vertex at the front of the priority queue (i.e. binary heap), see Figure c). This step results in no changes to the graph, so we move on to node $y$. At node $y$, see Figure d), we discover that it is cheaper to get to both $w$ and $z$ through $y$, so we adjust the distances and predecessor links accordingly. Finally we check nodes $w$ and $z$, see Figure e) and Figure f). However, no additional changes are found and so the priority queue is empty and Dijkstra’s algorithm exits.

![](./images/sp4.png)

It is important to note that Dijkstra’s algorithm works only when the weights are all positive. You should convince yourself that if you introduced a negative weight on one of the edges to the graph that the algorithm would never exit.

We will note that to route messages through the Internet, other algorithms are used for finding the shortest path. One of the problems with using Dijkstra’s algorithm on the Internet is that you must have a complete representation of the graph in order for the algorithm to run. The implication of this is that every router would need to have a complete map of all the routers in the Internet. In practice this is unfeasible and other variations of the algorithm allow each router to discover a partial graph Internet connectivity as they go. One such algorithm that you may want to read about is called the “distance vector” routing algorithm.

### Analysis of Dijkstra’s Algorithm

Finally, let us look at the running time of Dijkstra’s algorithm. We first note that building the priority queue takes $O(V)$ time since we initially add every vertex in the graph to the priority queue. Once the queue is constructed the `while` loop is executed once for every vertex since vertices are all added at the beginning and only removed after that. Within that loop each call to `delMin`, takes $O(logV)$ time. Taken together that part of the loop and the calls to `delMin` take $O(Vlog(V))$. The `for` loop is executed once for each edge in the graph, and within the `for` loop the call to `decreaseKey` takes time $O(Elog(V))$. So the combined running time is $O((V+E)log(V))$.

#### References

- [Problem Solving with Algorithms and Data Structures using Python by Bradley N. Miller, David L. Ranum is licensed under a Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License.](https://runestone.academy/runestone/books/published/pythonds/Graphs/toctree.html)